<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **ReAct: Build Reasoning and Acting AI Agents with LangGraph**


### Installing Required Libraries


In [ ]:
!pip install -U langgraph langchain-openai

In [ ]:
%%capture
!pip install langgraph==0.3.34 langchain-openai==0.3.14 langchainhub==0.1.21 langchain==0.3.24 pygraphviz==1.14 langchain-community==0.3.23

### Tools in ReAct

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
import os
import json

os.environ["TAVILY_API_KEY"] = "YOUR_KEY_HERE"

# Initialize the Tavily search tool
search = TavilySearchResults()

@tool
def search_tool(query: str):
    """
    Search the web for information using Tavily API.

    :param query: The search query string
    :return: Search results related to the query
    """
    return search.invoke(query)

In [ ]:
@tool
def recommend_clothing(weather: str) -> str:
    """
    Returns a clothing recommendation based on the provided weather description.

    This function examines the input string for specific keywords or temperature indicators
    (e.g., "snow", "freezing", "rain", "85°F") to suggest appropriate attire. It handles
    common weather conditions like snow, rain, heat, and cold by providing simple and practical
    clothing advice.

    :param weather: A brief description of the weather (e.g., "Overcast, 64.9°F")
    :return: A string with clothing recommendations suitable for the weather
    """
    weather = weather.lower()
    if "snow" in weather or "freezing" in weather:
        return "Wear a heavy coat, gloves, and boots."
    elif "rain" in weather or "wet" in weather:
        return "Bring a raincoat and waterproof shoes."
    elif "hot" in weather or "85" in weather:
        return "T-shirt, shorts, and sunscreen recommended."
    elif "cold" in weather or "50" in weather:
        return "Wear a warm jacket or sweater."
    else:
        return "A light jacket should be fine."

In [ ]:
tools=[search_tool,recommend_clothing]

tools_by_name={ tool.name:tool for tool in tools}

### Initializing the AI Model


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage,SystemMessage

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a helpful AI assistant that thinks step-by-step and uses tools when needed.

When responding to queries:
1. First, think about what information you need
2. Use available tools if you need current data or specific capabilities
3. Provide clear, helpful responses based on your reasoning and any tool results

Always explain your thinking process to help users understand your approach.
"""),
    MessagesPlaceholder(variable_name="scratch_pad")
])

In [ ]:
model_react=chat_prompt|model.bind_tools(tools)

#### Agent State


In [ ]:
from typing import (Annotated,Sequence,TypedDict)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """The state of the agent."""

    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
def tool_node(state: AgentState):
    """Execute all tool calls from the last message in the state."""
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}

#### Model Invocation Node


In [ ]:
def call_model(state: AgentState):
    """Invoke the model with the current conversation state."""
    response = model_react.invoke({"scratch_pad": state["messages"]})
    return {"messages": [response]}

In [ ]:
def should_continue(state: AgentState):
    """Determine whether to continue with tool use or end the conversation."""
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

In [ ]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Add edges between nodes
workflow.add_edge("tools", "agent")  # After tools, always go back to agent

# Add conditional logic
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",  # If tools needed, go to tools node
        "end": END,          # If done, end the conversation
    },
)

# Set entry point
workflow.set_entry_point("agent")

# Compile the graph
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
def print_stream(stream):
    """Helper function for formatting the stream nicely."""
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [HumanMessage(content="What's the weather like in Zurich, and what should I wear based on the temperature?")]}

print_stream(graph.stream(inputs, stream_mode="values"))

# Exercises

Now it's time to put your ReAct knowledge into practice! These exercises will help you build your own tools and extend the agent's capabilities.


## Exercise 1 - Build a Calculator Tool

**Objective:** Create a mathematical calculator tool that can handle complex calculations.

Your task is to create a calculator tool that can perform mathematical operations. This tool should be able to handle expressions like "2 + 3 * 4", "sqrt(16)", and "sin(π/2)".

### Instructions:
1. Create a tool called `calculator_tool` using the `@tool` decorator.
2. The tool should accept a mathematical expression as a string.
3. Use Python's `eval()` function carefully (or better yet, use the `ast` module for safety).
4. Test your tool with various mathematical expressions.
5. Add your tool to the tools list and test it with the ReAct agent.

### Starter Code:


In [ ]:
import math
import ast
import operator

@tool
def calculator_tool(expression: str) -> str:
    """
    Safely evaluate mathematical expressions.

    :param expression: A mathematical expression as a string (e.g., "2 + 3 * 4")
    :return: The result of the calculation
    """
    # Define safe operations
    safe_operators = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.Pow: operator.pow,
        ast.USub: operator.neg,
        ast.UAdd: operator.pos,
    }

    # Define safe functions
    safe_functions = {
        'abs': abs,
        'round': round,
        'min': min,
        'max': max,
        'sqrt': math.sqrt,
        'sin': math.sin,
        'cos': math.cos,
        'tan': math.tan,
        'log': math.log,
        'log10': math.log10,
        'exp': math.exp,
        'pi': math.pi,
        'e': math.e,
    }

    class SafeEvaluator(ast.NodeVisitor):
        def visit_BinOp(self, node):
            left = self.visit(node.left)
            right = self.visit(node.right)
            op = safe_operators.get(type(node.op))
            if op is None:
                raise ValueError(f"Unsafe operator: {type(node.op)}")
            return op(left, right)

        def visit_UnaryOp(self, node):
            operand = self.visit(node.operand)
            op = safe_operators.get(type(node.op))
            if op is None:
                raise ValueError(f"Unsafe operator: {type(node.op)}")
            return op(operand)

        def visit_Call(self, node):
            if not isinstance(node.func, ast.Name):
                raise ValueError("Only function names are allowed")

            func_name = node.func.id
            if func_name not in safe_functions:
                raise ValueError(f"Unsafe function: {func_name}")

            args = [self.visit(arg) for arg in node.args]
            return safe_functions[func_name](*args)

        def visit_Name(self, node):
            if node.id in safe_functions:
                return safe_functions[node.id]
            raise ValueError(f"Unsafe name: {node.id}")

        def visit_Constant(self, node):
            return node.value

        def visit_Num(self, node):
            return node.n

        def generic_visit(self, node):
            raise ValueError(f"Unsafe operation: {type(node)}")

    try:
        # Parse the expression
        tree = ast.parse(expression, mode='eval')

        # Evaluate safely
        evaluator = SafeEvaluator()
        result = evaluator.visit(tree.body)

        # Handle special cases
        if isinstance(result, (int, float)):
            # Format the result nicely
            if isinstance(result, int):
                return str(result)
            elif result.is_integer():
                return str(int(result))
            else:
                return f"{result:.6f}".rstrip('0').rstrip('.')
        else:
            return str(result)

    except Exception as e:
        return f"Error evaluating expression: {str(e)}"

# Add calculator_tool to your tools list
tools = [search_tool, recommend_clothing, calculator_tool]
tools_by_name = {tool.name: tool for tool in tools}

# Test with the agent: "What's 15% of 250 plus the square root of 144?"
inputs = {"messages": [HumanMessage(content="Calculate 15% of 250 plus the square root of 144")]}
print_stream(graph.stream(inputs, stream_mode="values"))

## Exercise 2 - Create a News Summary Tool

**Objective:** Build a tool that can fetch and summarize recent news articles.

Create a news summarization tool that works with the existing search functionality. This tool should take search results and create concise summaries of news articles.

### Instructions:
1. Create a `news_summarizer_tool` that takes news content and creates summaries.
2. The tool should extract key information: headline, date, main points.
3. Format the output in a readable way.
4. Test it by asking the agent to "search for recent AI news and summarize the top 3 articles".

### Starter Code:


In [ ]:
@tool
def news_summarizer_tool(news_content: str) -> str:
    """
    Summarize news articles from search results.

    :param news_content: Raw news content or search results
    :return: A formatted summary of the news
    """
    try:
        # Parse the news content - handle both JSON and string formats
        if news_content.strip().startswith('['):
            # Try to parse as JSON array
            articles = json.loads(news_content)
        else:
            # Handle as string content
            articles = [{"content": news_content, "title": "News Article", "url": ""}]

        if not articles:
            return "No news articles found to summarize."

        # Limit to top 3 articles
        articles = articles[:3]

        summary_parts = []
        summary_parts.append("�� **News Summary**\n")

        for i, article in enumerate(articles, 1):
            # Extract key information
            title = article.get('title', 'No title available')
            content = article.get('content', article.get('snippet', 'No content available'))
            url = article.get('url', 'No URL available')

            # Try to extract date if available
            date = article.get('published_date', 'Date not available')
            if date and date != 'Date not available':
                try:
                    # Try to format the date nicely
                    if isinstance(date, str):
                        # Handle various date formats
                        date_obj = datetime.fromisoformat(date.replace('Z', '+00:00'))
                        formatted_date = date_obj.strftime('%B %d, %Y')
                    else:
                        formatted_date = str(date)
                except:
                    formatted_date = str(date)
            else:
                formatted_date = "Date not available"

            # Extract main points (first 2-3 sentences)
            sentences = re.split(r'[.!?]+', content)
            main_points = '. '.join([s.strip() for s in sentences[:3] if s.strip()])
            if len(main_points) > 200:
                main_points = main_points[:200] + "..."

            # Format the article summary
            article_summary = f"""
                **Article {i}: {title}**
                Published: {formatted_date}
                Source: {url}

                **Summary:**
                {main_points}

                ---
            """
            summary_parts.append(article_summary)

        # Add overall summary
        total_articles = len(articles)
        summary_parts.append(f"\n📊 **Total Articles Summarized:** {total_articles}")

        return ''.join(summary_parts)

    except json.JSONDecodeError:
        # If JSON parsing fails, treat as plain text
        return f"""
            **News Summary**

            **Article Content:**
            {news_content[:500]}{'...' if len(news_content) > 500 else ''}

            **Note:** Content was processed as plain text due to parsing issues.
        """
    except Exception as e:
        return f"Error summarizing news: {str(e)}"

# Add news_summarizer_tool to your tools list
tools = [search_tool, recommend_clothing, calculator_tool, news_summarizer_tool]
tools_by_name = {tool.name: tool for tool in tools}

# Test with the agent
inputs = {"messages": [HumanMessage(content="Find recent AI news and summarize the top 3 articles")]}
print_stream(graph.stream(inputs, stream_mode="values"))

## Testing Your Solutions

For each exercise, test your implementation with these commands:


In [ ]:
# Exercise 1 Test
inputs = {"messages": [HumanMessage(content="Calculate 15% of 250 plus the square root of 144")]}
print_stream(graph.stream(inputs, stream_mode="values"))

In [ ]:
# Exercise 2 Test
inputs = {"messages": [HumanMessage(content="Find recent AI news and summarize the top 3 articles")]}
print_stream(graph.stream(inputs, stream_mode="values"))

Copyright © IBM Corporation. All rights reserved.
